Ordering
=====


Warning: This notebook will place live orders
==========================

Use a papertrading account when running this notebook.


In [1]:
%gui qt5

from ib_insync import *
util.useQt()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=13)

In [2]:
contract = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 100)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [3]:
trade = ib.placeOrder(contract, order)

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [4]:
trade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=13, action='BUY', totalQuantity=100), orderStatus=OrderStatus(status='Filled', filled=100.0, avgFillPrice=144.18, permId=798621671, lastFillPrice=144.18, clientId=13), fills=[Fill(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), execution=Execution(execId='00004466.5965a047.01.01', time='20170712  13:15:29', acctNumber='DU15065', exchange='ISLAND', side='BOT', shares=100.0, price=144.18, permId=798621671, clientId=13, orderId=13, cumQty=100.0, avgPrice=144.18), commissionReport=CommissionReport(execId='00004466.5965a047.01.01', commission=1.0, currency='USD', realizedPNL=1.7976931348623157e+308, yield_=1.7976931348623157e+308), time=datetime.datetime(2017, 7, 12, 11, 15, 29, 500675, tzinfo=datetime.timezone.utc))], lo

In [5]:
trade.log

[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 29, 242852), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 29, 389189, tzinfo=datetime.timezone.utc), status='Submitted', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 29, 500675, tzinfo=datetime.timezone.utc), status='Filled', message='Fill 100.0@144.18')]

``trade`` will also available from ``ib.trades()``,
but not from ``ib.openTrades()`` since it's already filled:

In [6]:
print(trade in ib.trades())
print(trade in ib.openTrades())

True
False


Likewise for ``order``:

In [7]:
print(order in ib.orders())
print(order in ib.openOrders())

True
False


Now let's create a limit order with an unrealistic limit:

In [8]:
limitOrder = LimitOrder('BUY', 100, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=14, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 38, 881335), status='PendingSubmit', message='')])

``status`` will change from "PendingSubmit" to "Submitted":

In [9]:
limitTrade.orderStatus.status

'Submitted'

In [10]:
limitTrade in ib.openTrades()

True

Let's modify the limit price and resubmit:

In [11]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=14, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=798621672, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 38, 881335), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 39, 12786, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 45, 115765), status='Submitted', message='Modify')])

And now cancel it:

In [12]:
ib.cancelOrder(limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=14, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=798621672, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 38, 881335), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 39, 12786, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 45, 115765), status='Submitted', message='Modify'), TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 48, 798617), status='PendingCancel', message='')])

In [13]:
limitTrade.log

[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 38, 881335), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 39, 12786, tzinfo=datetime.timezone.utc), status='Submitted', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 45, 115765), status='Submitted', message='Modify'),
 TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 48, 798617), status='PendingCancel', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 48, 916390, tzinfo=datetime.timezone.utc), status='Cancelled', message='')]

What's the total of commissions paid today?

In [14]:
sum(fill.commissionReport.commission for fill in ib.fills())

2.0

Some note of warning: reqContractDetails can happily report that a contract is unique,
but with placeOrder() saying that the contract is ambiguous. An example is INTC:

In [15]:
intc = Stock('INTC', 'SMART', 'USD')

print(len(ib.reqContractDetails(intc)))

intcOrder = LimitOrder('BUY', 100, 0.05)
ib.placeOrder(intc, intcOrder)

1


Trade(contract=Stock(symbol='INTC', exchange='SMART', currency='USD'), order=LimitOrder(orderId=16, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 58, 32645), status='PendingSubmit', message='')])

ERROR:ib_insync.wrapper:Error 200, reqId 16: The contract description specified for INTC is ambiguous.


In [18]:
ib.trades()[-1].log

[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 58, 32645), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 15, 58, 152491, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 200, reqId 16: The contract description specified for INTC is ambiguous.')]

So it is always a good
idea to a qualify a contract before trading it.

In [19]:
ib.qualifyContracts(intc)

intcOrder = LimitOrder('BUY', 100, 0.05)
ib.placeOrder(intc, intcOrder)

Trade(contract=Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), order=LimitOrder(orderId=18, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 16, 42, 117336), status='PendingSubmit', message='')])

In [20]:
ib.cancelOrder(intcOrder)

Trade(contract=Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), order=LimitOrder(orderId=18, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=798621673, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 16, 42, 117336), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 16, 42, 256663, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 12, 11, 16, 47, 876394), status='PendingCancel', message='')])